<a href="https://colab.research.google.com/github/anhpdd/ml-property-valuation-klang-valley/blob/main/notebooks/1_1__coord_to_wayid_manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OSM Way ID Extraction Tool
## Instructions:
1. Upload your CSV file (must have 'Coordinates' column)
2. Run all cells in order
3. Wait for processing (approx. 1 minute per 60 coordinates)
4. Download the results when complete

## Expected CSV Format:
- Must contain a 'Coordinates' column with format: "latitude, longitude"
- Example: "3.007702846839109, 101.46223641067644"

In [ ]:
import pandas as pd
import requests
import time
import json
from typing import Dict, Optional, Tuple
import warnings
from google.colab import files
import io
warnings.filterwarnings('ignore')

In [ ]:
print("\n" + "=" * 50)
print("STEP 1: Upload your CSV file")
print("=" * 50)
print("Please upload your CSV file with coordinates...")

uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"\n✓ File '{filename}' uploaded successfully!")

df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f"✓ Loaded {len(df)} rows from the file")

if 'Coordinates' not in df.columns:
    print("\n❌ ERROR: 'Coordinates' column not found!")
    print("Available columns:", df.columns.tolist())
    print("\nPlease ensure your CSV has a 'Coordinates' column")
else:
    print("✓ 'Coordinates' column found")
    print("\nFirst 5 rows of your data:")
    print(df.head())


STEP 1: Upload your CSV file
Please upload your CSV file with coordinates...


Saving tprop_df_validated.xlsx - Trang tính5 (1).csv to tprop_df_validated.xlsx - Trang tính5 (1) (1).csv

✓ File 'tprop_df_validated.xlsx - Trang tính5 (1) (1).csv' uploaded successfully!
✓ Loaded 1521 rows from the file
✓ 'Coordinates' column found

First 5 rows of your data:
                  road_name                             scheme_name  \
0        JALAN RESIDENCE 3A                          1080 RESIDENCE   
1        JALAN PERDANA 10/1  28 GREEN VIEW RESIDENCE @ SUNGAI PELEK   
2  JALAN SENTOSA (P/L 3021)  28 GREEN VIEW RESIDENCE @ SUNGAI PELEK   
3              JALAN 2/109C                             ABADI VILLA   
4     JALAN MAT KILAU 35/74                             ALAM IMPIAN   

                mukim      district  id  \
0              KAJANG   HULU LANGAT NaN   
1              SEPANG        SEPANG NaN   
2              SEPANG        SEPANG NaN   
3  MUKIM KUALA LUMPUR  KUALA LUMPUR NaN   
4              KELANG         KLANG NaN   

                              Coord

In [ ]:
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

BBOX = "(2.471,100.775,3.801,102.637)"

def parse_coordinate(coord_string: str) -> Tuple[float, float]:
    """
    Parse coordinate string "lat, lon" into tuple of floats
    """
    parts = coord_string.strip().split(',')
    lat = float(parts[0].strip())
    lon = float(parts[1].strip())
    return lat, lon

def create_overpass_query(lat: float, lon: float, radius: int = 50) -> str:
    """
    Create Overpass QL query to find nearest road within radius
    """
    query = f"""
    [out:json][timeout:25];
    (
      way["highway"]["highway"!~"footway|cycleway|pedestrian|path|steps|corridor|bridleway|proposed|construction|abandoned|platform|raceway"]
      (around:{radius},{lat},{lon})
      {BBOX};
    );
    out body;
    >;
    out skel qt;
    """
    return query

def get_distance_to_way(lat: float, lon: float, way_nodes: list, node_coords: dict) -> float:
    """
    Calculate approximate distance from point to way
    """
    import math

    min_dist = float('inf')

    for node_id in way_nodes:
        if node_id in node_coords:
            node_lat, node_lon = node_coords[node_id]
            dlat = math.radians(node_lat - lat)
            dlon = math.radians(node_lon - lon)
            a = math.sin(dlat/2)**2 + math.cos(math.radians(lat)) * math.cos(math.radians(node_lat)) * math.sin(dlon/2)**2
            c = 2 * math.asin(math.sqrt(a))
            dist = 6371000 * c
            min_dist = min(min_dist, dist)

    return min_dist

def query_nearest_road(lat: float, lon: float, retries: int = 3) -> Optional[Dict]:
    """
    Query Overpass API for nearest road to given coordinate
    """
    query = create_overpass_query(lat, lon)

    for attempt in range(retries):
        try:
            response = requests.post(
                OVERPASS_URL,
                data={'data': query},
                timeout=30
            )

            if response.status_code == 200:
                data = response.json()

                nodes = {}
                ways = []

                for element in data.get('elements', []):
                    if element['type'] == 'node':
                        nodes[element['id']] = (element['lat'], element['lon'])
                    elif element['type'] == 'way':
                        ways.append(element)

                if not ways:
                    return None

                nearest_way = None
                min_distance = float('inf')

                for way in ways:
                    dist = get_distance_to_way(lat, lon, way.get('nodes', []), nodes)
                    if dist < min_distance:
                        min_distance = dist
                        nearest_way = way

                if nearest_way:
                    return {
                        'way_id': nearest_way['id'],
                        'road_name': nearest_way.get('tags', {}).get('name', ''),
                        'road_type': nearest_way.get('tags', {}).get('highway', '')
                    }

                return None

            elif response.status_code == 429:
                time.sleep(2 ** attempt)
            else:
                time.sleep(1)

        except requests.exceptions.Timeout:
            time.sleep(2)
        except Exception as e:
            time.sleep(2)

    return None

print("✓ Helper functions defined")


✓ Helper functions defined


In [ ]:
print("\n" + "=" * 50)
print("STEP 2: Processing Coordinates")
print("=" * 50)

df['way_id'] = None
df['osm_road_name'] = None
df['road_type'] = None

total_rows = len(df)
successful = 0
failed = 0
failed_indices = []

print(f"Starting to process {total_rows} coordinates...")
print(f"Estimated time: {total_rows * 1.2 / 60:.1f} minutes")
print("\nProgress:")

from IPython.display import clear_output

for idx, row in df.iterrows():
    current = idx + 1
    percentage = current / total_rows * 100

    if current % 5 == 0 or current == total_rows:
        clear_output(wait=True)
        print("STEP 2: Processing Coordinates")
        print("=" * 50)
        print(f"Progress: {current}/{total_rows} ({percentage:.1f}%)")

        bar_length = 40
        filled_length = int(bar_length * current // total_rows)
        bar = '█' * filled_length + '-' * (bar_length - filled_length)
        print(f"[{bar}]")

        print(f"\n✓ Successful: {successful}")
        print(f"✗ Failed: {failed}")

        if current < total_rows:
            remaining_time = (total_rows - current) * 1.2 / 60
            print(f"\nEstimated time remaining: {remaining_time:.1f} minutes")

    try:
        lat, lon = parse_coordinate(row['Coordinates'])

        result = query_nearest_road(lat, lon)

        if result:
            df.at[idx, 'way_id'] = result['way_id']
            df.at[idx, 'osm_road_name'] = result['road_name']
            df.at[idx, 'road_type'] = result['road_type']
            successful += 1
        else:
            failed += 1
            if 'index' in df.columns:
                failed_indices.append(row['index'])
            else:
                failed_indices.append(idx)

        time.sleep(1)

    except Exception as e:
        failed += 1
        if 'index' in df.columns:
            failed_indices.append(row['index'])
        else:
            failed_indices.append(idx)
        continue

print("\n" + "=" * 50)
print("✓ Processing Complete!")
print("=" * 50)


STEP 2: Processing Coordinates
Progress: 1521/1521 (100.0%)
[████████████████████████████████████████]

✓ Successful: 1489
✗ Failed: 31

✓ Processing Complete!


In [ ]:
print("\n📊 RESULTS SUMMARY")
print("=" * 50)
print(f"Total rows processed: {total_rows}")
print(f"✓ Successful: {successful} ({successful/total_rows*100:.1f}%)")
print(f"✗ Failed: {failed} ({failed/total_rows*100:.1f}%)")

print(f"\nRows with way_id: {df['way_id'].notna().sum()}")
print(f"Rows with road name: {df['osm_road_name'].notna().sum()}")
print(f"Unnamed roads (no name in OSM): {(df['way_id'].notna() & (df['osm_road_name'] == '')).sum()}")

print("\n📈 Road Type Distribution:")
road_type_counts = df['road_type'].value_counts()
for road_type, count in road_type_counts.head(10).items():
    print(f"  {road_type}: {count}")

print("\n🔍 Sample Results (first 5 successful):")
sample_cols = ['way_id', 'osm_road_name', 'road_type']
if 'index' in df.columns:
    sample_cols = ['index'] + sample_cols
print(df[df['way_id'].notna()][sample_cols].head())

if failed_indices and len(failed_indices) > 0:
    print(f"\n⚠️ Failed indices: {failed_indices[:10]}{'...' if len(failed_indices) > 10 else ''}")


📊 RESULTS SUMMARY
Total rows processed: 1521
✓ Successful: 1490 (98.0%)
✗ Failed: 31 (2.0%)

Rows with way_id: 1490
Rows with road name: 1490
Unnamed roads (no name in OSM): 534

📈 Road Type Distribution:
  residential: 1229
  service: 173
  unclassified: 39
  tertiary: 33
  track: 5
  secondary: 4
  primary: 4
  trunk: 2
  motorway: 1

🔍 Sample Results (first 5 successful):
       way_id           osm_road_name    road_type
0  1335883496       Jalan Residence 4  residential
1  1010284500                          residential
2   744568149                          residential
3    39900028            Jalan 3/109C  residential
4   856851365  Jalan Mat Kilau 35/74A  residential

⚠️ Failed indices: [114, 240, 324, 431, 432, 526, 527, 528, 529, 786]...


In [ ]:
print("\n" + "=" * 50)
print("STEP 3: Download Results")
print("=" * 50)

output_filename = filename.replace('.csv', '_with_wayids.csv')
df.to_csv(output_filename, index=False)
print(f"✓ Results saved to: {output_filename}")

if failed > 0:
    failed_df = df[df['way_id'].isna()]
    failed_filename = filename.replace('.csv', '_failed.csv')
    failed_df.to_csv(failed_filename, index=False)
    print(f"✓ Failed rows saved to: {failed_filename}")

print("\n📥 Downloading files...")

files.download(output_filename)

if failed > 0:
    files.download(failed_filename)
    print(f"\n⚠️ Please review the {failed} failed rows in '{failed_filename}'")
    print("These coordinates couldn't find roads within 50m radius")

print("\n" + "=" * 50)
print("✅ ALL DONE!")
print("=" * 50)
print(f"Your file '{output_filename}' has been downloaded.")


STEP 3: Download Results
✓ Results saved to: tprop_df_validated.xlsx - Trang tính5 (1) (1)_with_wayids.csv
✓ Failed rows saved to: tprop_df_validated.xlsx - Trang tính5 (1) (1)_failed.csv

📥 Downloading files...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


⚠️ Please review the 31 failed rows in 'tprop_df_validated.xlsx - Trang tính5 (1) (1)_failed.csv'
These coordinates couldn't find roads within 50m radius

✅ ALL DONE!
Your file 'tprop_df_validated.xlsx - Trang tính5 (1) (1)_with_wayids.csv' has been downloaded.


In [ ]:
if failed > 0:
    print("\n" + "=" * 50)
    print("Failed Rows Analysis (Optional)")
    print("=" * 50)

    failed_df = df[df['way_id'].isna()]

    print(f"\nShowing details of {min(10, len(failed_df))} failed rows:")
    display_cols = ['Coordinates']
    if 'index' in df.columns:
        display_cols = ['index'] + display_cols
    if 'road_name' in df.columns:
        display_cols = display_cols + ['road_name']

    print(failed_df[display_cols].head(10))

    print("\nPossible reasons for failures:")
    print("1. No road within 50m radius")
    print("2. Coordinate outside the bounding box")
    print("3. Temporary API issues")
    print("\nYou may need to manually verify these coordinates")